In [1]:
from dotenv import load_dotenv
from utils.agent_visualizer import print_activity

from claude_code_sdk import ClaudeCodeOptions, ClaudeSDKClient, query

load_dotenv()

True

# 00 - The One-Liner Research Agent

PREFACE: We highly recommend reading [Building effective agents](https://www.anthropic.com/engineering/building-effective-agents) or [How we built our multi-agent research system](https://www.anthropic.com/engineering/built-multi-agent-research-system) in case you haven't. They are great reads and we will assume some basic understanding of agents! 

In this notebook we build our own (re)search agent, which is inherently a great use-case because of a few reasons:
- The input to our system is not sufficient to produce an output, meaning there needs to be interaction with external systems (e.g., the internet)
- There is no predefined workflow we can use since it is unclear what the agent will discover during its research

Instead, a research agent requires the flexibility to explore unexpected leads and change direction based on what it finds. In its simplest form, a research agent can be an agent that simply searches the internet and summarizes it for you. 

Below, we'll implement a basic research agent with just a few lines of code. We provide Claude with exactly one tool which the Claude Code SDK contains straight out of the box: [web search tool](https://docs.claude.com/en/docs/agents-and-tools/tool-use/web-search-tool). 

> Check [here](https://docs.claude.com/en/docs/claude-code/settings#tools-available-to-claude) for a list of Claude Code's readily available tools

In [2]:
messages = []
async for msg in query(
    prompt="Research the latest trends in AI agents and give me a brief summary",
    options=ClaudeCodeOptions(model="claude-sonnet-4-20250514", allowed_tools=["WebSearch"]),
):
    print_activity(msg)
    messages.append(msg)

🤖 Thinking...
🤖 Using: WebSearch()
✓ Tool completed
🤖 Thinking...


In [ ]:
print(
    f"\nResult:\n{messages[-1].result if hasattr(messages[-1], 'result') and messages[-1].result else messages[-2].content[0].text}"
)

And that's all it takes! Just like that we have a research agent that can go and browse the web to answer (to the best of its ability, at least) any question you throw at it.

Note that in our query we provided the argument `options`. Here we define the configuration, the capabilities and limitations of our agent. For example, we provide our agent with the ability to search the web by passing ```allowed_tool=["WebSearch"]```.

More specifically, `allowed_tools` is a list of tools that Claude will be able to use without any approvals. The rest of the tools are still available, but Claude will ask for approval to use them. That said, certain tools like `Read` and other base read-only tools are always allowed. If you want any tool to be removed from Claude's context, add it to `disallowed_tools` instead.

Now, to more closely inspect the actions our agent took, we have provided the ```visualize_conversation``` function.

In [ ]:
from utils.agent_visualizer import visualize_conversation

visualize_conversation(messages)

### Supercharging our agent

So far, we have laid out a very simple (maybe naive) implementation to illustrate how you can start leveraging the SDK to build a research agent. However, there are various ways we can improve our agent to turn it production ready. Let's cover a few of them:

1. Notice how before we only sent one query? In many systems, a human will look at the output of the system, potentially assigning a follow up task. Just like text completions, if we want to send multiple queries to the agent (e.g., 1. analyze abc, 2. make xyz based on your analysis) we would have to copy over the entire analysis context in our second query. Instead, we can **[use the ClaudeSDKClient](https://docs.claude.com/en/docs/claude-code/sdk/sdk-python#1-the-claudesdkclient-class-recommended)** to maintain the conversation context for us.

2. Another great way of steering the system is **providing a system prompt**, akin to a system prompt used for text completions. To learn how to write a good system prompt for a research agent, we recommend looking [here](https://github.com/anthropics/anthropic-cookbook/tree/main/patterns/agents/prompts).

3. **Leveraging the `Read` tool** to enable multimodal input. This tool allows Claude to analyze charts, infographics, and complex system diagrams.

In [5]:
messages = []
async with ClaudeSDKClient(
    options=ClaudeCodeOptions(
        model="claude-sonnet-4-20250514",
        cwd="research_agent",
        system_prompt="You are a research agent specialized in AI",
        allowed_tools=["WebSearch", "Read"],
    )
) as research_agent:
    await research_agent.query("Analyze the chart in research_agent/projects_claude.png")
    async for msg in research_agent.receive_response():
        print_activity(msg)
        messages.append(msg)

    await research_agent.query("Use a single websearch to investigate the insights from the chart.")
    async for msg in research_agent.receive_response():
        print_activity(msg)
        messages.append(msg)

🤖 Thinking...
🤖 Using: Read()
✓ Tool completed
🤖 Thinking...
🤖 Using: Bash()
✓ Tool completed
🤖 Thinking...
🤖 Using: Read()
✓ Tool completed
🤖 Thinking...
🤖 Thinking...
🤖 Using: WebSearch()
✓ Tool completed
🤖 Thinking...


In [ ]:
visualize_conversation(messages)

### The Research Agent leaves Jupyter

Finally, to be able to use the agent outside our notebook, we must put it in a Python script. A lightweight implementation of our research agent can be found in `research_agent/agent.py`. We define three functions:
- `print_activity()` - Shows what the agent is doing in real-time
- `get_activity_text()` - Extracts activity text for custom handlers
- `send_query()` - Main function for sending and handlingqueries with built-in activity display

This agent can now be used in any Python script!

First an example to test a one-off query to the agent:

In [ ]:
from research_agent.agent import send_query

result = await send_query("What is the Claude Code SDK? Only do one websearch and be concise")
print(f"\nResult: {result}\n")

Now we test out a multi-turn conversation that reuses the same conversation:

In [8]:
result1 = await send_query("What is Anthropic? Only do one websearch and be concise")
print(f"\n-----\n\nInitial research: {result1}\n")

🤖 Thinking...
🤖 Using: WebSearch()
✓ Tool completed
🤖 Thinking...

-----

Initial research: Anthropic is an AI safety and research company founded in 2021 by former OpenAI researchers, including siblings Dario and Daniela Amodei. The company develops Claude, a family of large language models (LLMs) designed to be helpful, harmless, and honest.

**Key points:**
- **Mission**: Build reliable, interpretable, and steerable AI systems with a focus on AI safety
- **Main product**: Claude AI assistant (which you're currently using!)
- **Structure**: Public benefit corporation balancing profit with humanity's long-term benefit
- **Funding**: Backed by major investments from Amazon ($8B total) and Google ($2B)
- **Focus areas**: AI safety, natural language processing, human feedback, and responsible AI development

Anthropic positions itself as a "safety-first" AI lab, emphasizing the responsible development of AI systems to serve humanity's long-term well-being.



In [ ]:
# Continue the conversation to dig deeper by setting continue_conversation=True
result2 = await send_query(
    "What are some of their products?",
    continue_conversation=True,
)
print(f"\n-----\n\nFollow-up: {result2}\n")

## Conclusion

We've demonstrated how the Claude Code SDK enables you to build a functional research agent in just a few lines of code. By leveraging the built-in WebSearch tool, we created an agent capable of autonomous information gathering and synthesis. We also explored how the
ClaudeSDKClient maintains conversation context across multiple queries and how to incorporate multimodal capabilities through the Read tool.

This foundation in basic agentic workflows prepares you for more sophisticated implementations. In the next notebook, we'll advance to building a Chief of Staff agent that coordinates multiple specialized subagents, implements custom output styles for different
stakeholders, and uses hooks for governance and compliance tracking.

Next: [01_The_chief_of_staff_agent.ipynb](01_The_chief_of_staff_agent.ipynb) - Learn how to orchestrate complex multi-agent systems with enterprise-grade features.
